# Introduction
* description: Example jupyter notebook for loading Tidepool data and calculating cgm stats in R
* version: 0.0.1
* created: 2018-07-17
* author: Jason Meno
* dependencies:
    * requires tidepool-analytics environment (see root directory for instructions
    on how to install the environment).
* license: BSD-2-Clause

# Load data

In [136]:
# Set location of data
dataPath = "../example-data/"

# load csv data
data = read.csv(paste0(dataPath,
                       "example-from-j-jellyfish.csv"),
                       stringsAsFactors = FALSE)

# View the first 5 rows of data
head(data)

jsonRowIndex,annotations,clockDriftOffset,conversionOffset,deliveryType,deviceId,deviceTime,duration,expectedDuration,guid,...,state,timeProcessing,timezone,version,bgInput,bolus,carbInput,insulinCarbRatio,insulinOnBoard,recommended
2193,,0,0,scheduled,InsOmn-130346997,2017-12-31T18:00:00,14400000,NA,6b6b6e42-abcf-4554-b121-7074a0918484,...,,,,,NA,,NA,NA,NA,
2190,,0,0,scheduled,InsOmn-130346997,2017-12-31T22:00:00,7200000,NA,cc62b7b6-227c-4bef-9883-b88066063706,...,,,,,NA,,NA,NA,NA,
2188,,0,0,scheduled,InsOmn-130346997,2018-01-01T00:00:00,12600000,NA,5f37715a-3589-4283-8d88-f9398f14c2d3,...,,,,,NA,,NA,NA,NA,
2185,,0,0,scheduled,InsOmn-130346997,2018-01-01T03:30:00,7200000,NA,f9aeb728-6723-4f3d-8a03-cd8854a459bb,...,,,,,NA,,NA,NA,NA,
2184,,0,0,scheduled,InsOmn-130346997,2018-01-01T05:30:00,10163000,NA,bab3ca8e-f663-4299-a1c9-60b5b766a868,...,,,,,NA,,NA,NA,NA,
2182,,0,0,suspend,InsOmn-130346997,2018-01-01T08:19:23,227000,NA,bb5109fc-9cf9-46a7-abbd-2d375337b91c,...,,,,,NA,,NA,NA,NA,


In [177]:
# Get a list of all data columns
colnames(data)

[1] "jsonRowIndex"        "annotations"         "clockDriftOffset"   
 [4] "conversionOffset"    "deliveryType"        "deviceId"           
 [7] "deviceTime"          "duration"            "expectedDuration"   
[10] "guid"                "id"                  "payload"            
[13] "percent"             "rate"                "scheduleName"       
[16] "suppressed"          "time"                "timezoneOffset"     
[19] "type"                "uploadId"            "expectedNormal"     
[22] "extended"            "normal"              "subType"            
[25] "X_deduplicator"      "units"               "value"              
[28] "alarmType"           "reason"              "status"             
[31] "activeSchedule"      "basalSchedules"      "bgTarget"           
[34] "carbRatio"           "insulinSensitivity"  "X_dataState"        
[37] "X_state"             "byUser"              "computerTime"       
[40] "dataSetType"         "deviceManufacturers" "deviceModel"        
[43] "deviceSerialNumber"  "deviceTags"          "state"              
[46] "timeProcessing"      "timezone"            "version"            
[49] "bgInput"             "bolus"               "carbInput"          
[52] "insulinCarbRatio"    "insulinOnBoard"      "recommended"

In [176]:
# Get the unique data types
unique(data$type)

[1] "basal"        "bolus"        "cbg"          "deviceEvent"  "pumpSettings"
[6] "smbg"         "upload"       "wizard"

# Get cgm stats

In [139]:
# get just the cgm data
cgm = data[data$type=="cbg",]

# look at the first 5 rows of data
head(cgm)

,jsonRowIndex,annotations,clockDriftOffset,conversionOffset,deliveryType,deviceId,deviceTime,duration,expectedDuration,guid,...,state,timeProcessing,timezone,version,bgInput,bolus,carbInput,insulinCarbRatio,insulinOnBoard,recommended
714,1933,,0,0,,AbbottFreeStyleLibre-JKGX280-T2691,2018-01-17T02:51:17,NA,NA,9ccd402e2a9c4275b5ac226d1dc73564,...,,,,,NA,,NA,NA,NA,
715,1932,,0,0,,AbbottFreeStyleLibre-JKGX280-T2691,2018-01-17T03:06:17,NA,NA,af423d055ecf4bc09864f56705f0c246,...,,,,,NA,,NA,NA,NA,
716,1931,,0,0,,AbbottFreeStyleLibre-JKGX280-T2691,2018-01-17T03:21:17,NA,NA,d8efb6c8c88a4b35a2e8a34480d81426,...,,,,,NA,,NA,NA,NA,
717,1929,,0,0,,AbbottFreeStyleLibre-JKGX280-T2691,2018-01-17T03:36:17,NA,NA,f4c6f77b38964eb991c189ea4b4dd3e7,...,,,,,NA,,NA,NA,NA,
718,1928,,0,0,,AbbottFreeStyleLibre-JKGX280-T2691,2018-01-17T03:51:17,NA,NA,355075ac8d7b42c3b4fba5992aefd593,...,,,,,NA,,NA,NA,NA,
719,1927,,0,0,,AbbottFreeStyleLibre-JKGX280-T2691,2018-01-17T04:06:17,NA,NA,44b40c20cb134dcbb9eba080e1bbac64,...,,,,,NA,,NA,NA,NA,


In [140]:
# rename "value" field to "mmol_L"
colnames(cgm)[colnames(cgm)=="value"] = "mmol_L"

# convert mmol/L to mg/dL and create a new field
cgm$mg_dL = as.integer(cgm$mmol_L * 18.01559)

# view the cgm mg/dL data
head(cgm$mg_dL)

[1]  70  85  94 102 102  97

In [174]:
# define a function that captures the Ambulatory Glucose Profile statistics
# http://www.agpreport.org/agp/agpreports#CGM_AGP
get_stats <- function(df){
    
    statsDF = data.frame(matrix(nrow=1,ncol=0))
    
    totalNumberCBGValues = length(df$mg_dL)
    statsDF$totalCgmValues = totalNumberCBGValues
    
    df$deviceTime = as.POSIXct(df$deviceTime,format="%Y-%m-%dT%H:%M:%S")
    firstDataPoint = min(df$deviceTime)
    lastDataPoint = max(df$deviceTime)
    
    if(grepl("FreeStyle", df$deviceId[1])==TRUE){
        dataFrequency = 15
    }else{
        dataFrequency = 5
    }
     
    totalPossibleCgmValues = round(as.integer(difftime(lastDataPoint, 
                                    firstDataPoint,
                                    units="min"))/dataFrequency)

    
    statsDF$totalPossibleCgmReadings = totalPossibleCgmValues
        
    statsDF$percentOfPossibleCgmReadings = 
                   totalNumberCBGValues / totalPossibleCgmValues
    
    statsDF$firstDataPoint = firstDataPoint 
    statsDF$lastDataPoint = lastDataPoint
    statsDF$daysOfCgmData = as.integer(
            difftime(as.Date(lastDataPoint), 
                     as.Date(firstDataPoint),
                     units="days"))
      
    mean_mgdL = mean(df$mg_dL)
    statsDF$mean_mgdL = mean_mgdL
    
    std_mgdL = sd(df$mg_dL)
    statsDF$std_mgdL = std_mgdL
    
    cov_mgdL = std_mgdL / mean_mgdL
    statsDF$cov_mgdL = cov_mgdL
    
    totalBelow54 = sum(df$mg_dL < 54)
    statsDF$percentBelow54 = totalBelow54 / totalNumberCBGValues
    
    totalBelow70 = sum(df$mg_dL < 70)    
    statsDF$percentBelow70 = totalBelow70 / totalNumberCBGValues
    
    total70to180 = sum((df$mg_dL >= 70) & (df$mg_dL <= 180))    
    statsDF$percentTimeInRange = total70to180 / totalNumberCBGValues
    
    totalAbove180 = sum(df$mg_dL > 180)
    statsDF$percentAbove180 = totalAbove180 / totalNumberCBGValues
    
    totalAbove250 = sum(df$mg_dL > 250)
    statsDF$percentAbove250 = totalAbove250 / totalNumberCBGValues

    statsDF$min_mgdL = min(df$mg_dL)
    statsDF$`10%` = quantile(df$mg_dL,0.10)
    statsDF$`25%` = quantile(df$mg_dL,0.25)
    statsDF$median = quantile(df$mg_dL,0.50)
    statsDF$`75%` = quantile(df$mg_dL,0.75)
    statsDF$`90%` = quantile(df$mg_dL,0.90)
    statsDF$max_mgdL = max(df$mg_dL)

    # get estimated HbA1c or Glucose Management Index (GMI)
    # GMI(%) = 3.31 + 0.02392 x [mean glucose in mg/dL]
    # https://www.jaeb.org/gmi/
    statsDF$GMI = 3.31 + (0.02392 * mean_mgdL)
    
    return(statsDF)
    
}

In [178]:
# apply function to get stats
cgmStats = get_stats(cgm)
t(cgmStats)

totalCgmValues,1282
totalPossibleCgmReadings,1367
percentOfPossibleCgmReadings,0.93782
firstDataPoint,2018-01-17 02:51:17
lastDataPoint,2018-01-31 08:34:03
daysOfCgmData,14
mean_mgdL,137.7902
std_mgdL,57.96422
cov_mgdL,0.4206702
percentBelow54,0.02340094
percentBelow70,0.08268331
